# Chapter 4: SQL & BigQuery

## Author: Audrey Marthin

### Sources: 
- https://pandas.pydata.org/docs/ 
- Professor Wirfs-Brock's Class Code Demo
- Audrey's projects

Date: Sunday, December 10

In [47]:
import pandas as pd
import numpy as np

### Part 1: Process the Data before Uploading to BigQuery
This is an optional step, you can also upload directly or use datasets from BigQuery. But, I am processing here to re-format some of the data values.

In [48]:
# import csv as dataframe
plastic_emitted_ocean = pd.read_csv("data_bigquery/plastic-waste-emitted-to-the-ocean.csv")
global_mismanaged_share = pd.read_csv("data_bigquery/share-of-global-mismanaged-plastic-waste.csv")
global_emitted_share = pd.read_csv("data_bigquery/share-of-global-plastic-waste-emitted-to-the-ocean.csv")
global_gdp = pd.read_csv("data_bigquery/global_information.csv")

In [49]:
# rename the country code column to code to match up and also remove codes from column names of GDP data
global_gdp.rename(columns={"Country Code": "Code", "Population, total [SP.POP.TOTL]": "Total Population",
                           "Surface area (sq. km) [AG.SRF.TOTL.K2]": "Surface Area (sq. km)",
                            "CO2 emissions (metric tons per capita) [EN.ATM.CO2E.PC]": "CO2 emission (metric tons per capita)",	
                            "GDP (current US$) [NY.GDP.MKTP.CD]": "GDP (current US$)",
                            "GDP growth (annual %) [NY.GDP.MKTP.KD.ZG]": "GDP growth (annual%)",	
                            "GDP per capita (current US$) [NY.GDP.PCAP.CD]": "GDP per capita (current US$)",	
                            "Population density (people per sq. km of land area) [EN.POP.DNST]": "Population density (people per sq. km)"}, inplace=True)

# dropna for time and examine
global_gdp.dropna(subset=["Time", "Country Name"], inplace=True)
global_gdp.drop(columns=["Time Code"], inplace=True)

In [50]:
# Examine
global_gdp


,Time,Country Name,Code,Total Population,Surface Area (sq. km),CO2 emission (metric tons per capita),GDP (current US$),GDP growth (annual%),GDP per capita (current US$),Population density (people per sq. km)
0,2019,Afghanistan,AFG,37769499.0,652860,0.297563650500103,18904502222.2108,3.911603419423,500.52298078433,57.9082516903546
1,2019,Albania,ALB,2854191.0,28750,1.74946245713759,15401826080.5183,2.08771199649604,5396.21422690995,104.167554744526
2,2019,Algeria,DZA,42705368.0,2381741,3.99440182789199,171760290068.157,0.999999999741789,4021.98360796603,17.9303156808402
3,2019,American Samoa,ASM,47321.0,200,..,647000000,-0.487804878048777,13672.576657298,236.605
4,2019,Andorra,AND,76343.0,470,6.2872038038851,3155149347.92151,2.01554763848308,41328.6004993451,162.431914893617
...,...,...,...,...,...,...,...,...,...,...
212,2019,Virgin Islands (U.S.),VIR,106669.0,350,..,4117000000,2.77777777777777,38596.0307118282,304.768571428571
213,2019,West Bank and Gaza,PSE,4685306.0,6020,..,17133500000,1.36268746558062,3656.85827137011,778.290033222591
214,2019,"Yemen, Rep.",YEM,31546691.0,527970,0.354864476911382,..,..,693.816483547342,59.7509157717295
215,2019,Zambia,ZMB,18380477.0,752610,0.414336363523101,23308667781.2258,1.44130602603785,1268.12094056241,24.7252142213374


In [52]:
# Check the dataframe for values
global_gdp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 217 entries, 0 to 216
Data columns (total 10 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Time                                    217 non-null    object 
 1   Country Name                            217 non-null    object 
 2   Code                                    217 non-null    object 
 3   Total Population                        217 non-null    float64
 4   Surface Area (sq. km)                   217 non-null    object 
 5   CO2 emission (metric tons per capita)   217 non-null    object 
 6   GDP (current US$)                       217 non-null    object 
 7   GDP growth (annual%)                    217 non-null    object 
 8   GDP per capita (current US$)            217 non-null    object 
 9   Population density (people per sq. km)  217 non-null    object 
dtypes: float64(1), object(9)
memory usage: 18.6+ KB


In [53]:
# Some columns are not in float values type, still in string
columns_to_convert = ["Total Population", "Surface Area (sq. km)", "CO2 emission (metric tons per capita)",	
                      "GDP (current US$)", "GDP growth (annual%)", "GDP per capita (current US$)", "Population density (people per sq. km)"]

# Replace non-numeric values with NaN
global_gdp[columns_to_convert] = global_gdp[columns_to_convert].replace('..', np.nan)

# Convert specified columns to float
global_gdp[columns_to_convert] = global_gdp[columns_to_convert].astype(float)
global_gdp


,Time,Country Name,Code,Total Population,Surface Area (sq. km),CO2 emission (metric tons per capita),GDP (current US$),GDP growth (annual%),GDP per capita (current US$),Population density (people per sq. km)
0,2019,Afghanistan,AFG,37769499.0,652860.0,0.297564,1.890450e+10,3.911603,500.522981,57.908252
1,2019,Albania,ALB,2854191.0,28750.0,1.749462,1.540183e+10,2.087712,5396.214227,104.167555
2,2019,Algeria,DZA,42705368.0,2381741.0,3.994402,1.717603e+11,1.000000,4021.983608,17.930316
3,2019,American Samoa,ASM,47321.0,200.0,NaN,6.470000e+08,-0.487805,13672.576657,236.605000
4,2019,Andorra,AND,76343.0,470.0,6.287204,3.155149e+09,2.015548,41328.600499,162.431915
...,...,...,...,...,...,...,...,...,...,...
212,2019,Virgin Islands (U.S.),VIR,106669.0,350.0,NaN,4.117000e+09,2.777778,38596.030712,304.768571
213,2019,West Bank and Gaza,PSE,4685306.0,6020.0,NaN,1.713350e+10,1.362687,3656.858271,778.290033
214,2019,"Yemen, Rep.",YEM,31546691.0,527970.0,0.354864,NaN,NaN,693.816484,59.750916
215,2019,Zambia,ZMB,18380477.0,752610.0,0.414336,2.330867e+10,1.441306,1268.120941,24.725214


In [51]:
# merge the plastic management global country data 
df_global_plastic_management = pd.merge(plastic_emitted_ocean, 
                               global_mismanaged_share[["Entity", "Year", "Share of global mismanaged plastic waste"]], 
                               on=["Entity", "Year"], how="left").merge(
                               global_emitted_share[["Entity", "Year", "Share of global plastics emitted to ocean"]], 
                               on=["Entity", "Year"], how="left")
df_global_plastic_management

,Entity,Code,Year,Mismanaged waste emitted to the ocean (metric tons year-1),Share of global mismanaged plastic waste,Share of global plastics emitted to ocean
0,Africa,NaN,2019,78252,22.161703,7.989317
1,Albania,ALB,2019,1565,0.113046,0.159782
2,Algeria,DZA,2019,5774,1.237708,0.589510
3,Angola,AGO,2019,860,0.383571,0.087804
4,Antigua and Barbuda,ATG,2019,2,0.001015,0.000204
...,...,...,...,...,...,...
166,Vietnam,VNM,2019,28221,1.801398,2.881287
167,Western Sahara,ESH,2019,38,0.006660,0.003880
168,World,OWID_WRL,2019,979458,NaN,NaN
169,Yemen,YEM,2019,252,0.472267,0.025729


In [59]:
# Drop rows where code is Null so that we only get the country data
df_global_plastic_management.dropna(subset=["Code"], inplace=True)

# Drop the world data
df_global_plastic_management.drop(df_global_plastic_management[df_global_plastic_management["Code"] == "OWID_WRL"].index, inplace=True)
df_global_plastic_management

,Entity,Code,Year,Mismanaged waste emitted to the ocean (metric tons year-1),Share of global mismanaged plastic waste,Share of global plastics emitted to ocean
1,Albania,ALB,2019,1565,0.113046,0.159782
2,Algeria,DZA,2019,5774,1.237708,0.589510
3,Angola,AGO,2019,860,0.383571,0.087804
4,Antigua and Barbuda,ATG,2019,2,0.001015,0.000204
5,Argentina,ARG,2019,4137,0.754056,0.422376
...,...,...,...,...,...,...
165,Venezuela,VEN,2019,5988,1.086920,0.611359
166,Vietnam,VNM,2019,28221,1.801398,2.881287
167,Western Sahara,ESH,2019,38,0.006660,0.003880
169,Yemen,YEM,2019,252,0.472267,0.025729


In [54]:
# Check for values
df_global_plastic_management.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171 entries, 0 to 170
Data columns (total 6 columns):
 #   Column                                                      Non-Null Count  Dtype  
---  ------                                                      --------------  -----  
 0   Entity                                                      171 non-null    object 
 1   Code                                                        160 non-null    object 
 2   Year                                                        171 non-null    int64  
 3   Mismanaged waste emitted to the ocean (metric tons year-1)  171 non-null    int64  
 4   Share of global mismanaged plastic waste                    170 non-null    float64
 5   Share of global plastics emitted to ocean                   170 non-null    float64
dtypes: float64(2), int64(2), object(2)
memory usage: 8.1+ KB


In [62]:
# Save the DataFrames to a CSV file
global_gdp.to_csv("data_bigquery/global_gdp_co2.csv", index=False)
df_global_plastic_management.to_csv("data_bigquery/global_plastic_management.csv", index=False)

### Part 2: SQL and BigQuery


Steps:
- Create a BigQuery account (https://cloud.google.com/bigquery/docs/introduction)
- Create a new project
- Create a dataset within the project
- Create a table within dataset
- Start making queries!

I will show some sample queries in this notebook, check out the link to the actual queries as well!

#### Find Countries with Least Share of Global Mismanaged Plastic Waste
Query Link = https://console.cloud.google.com/bigquery?sq=183187291487:e70146285e054b7cbc6cb0d97a12fa25

Explanation =
- Filter the global_plastic_management datasets where the share of global mismanaged plastic waste * 100 (changing this to percentage) is < 5%
- Order in ascending order, so smallest to largest value

SELECT Entity, Code, Year, Share_of_global_mismanaged_plastic_waste FROM `project-10-sql-and-bigquery.datasets.global_plastic_management`

WHERE Share_of_global_mismanaged_plastic_waste*100 < 5

ORDER BY Share_of_global_mismanaged_plastic_waste ASC

LIMIT 1000;

#### Find Countries with CO2 Emissions per Capita greater than 10 Tons per Capita
Query Link = https://console.cloud.google.com/bigquery?sq=183187291487:eaa17d7f20914e2280b61a4fad1691b7

Explanation =
- Filter global_gdp_co2 for CO2 emission metric tons per capita is larger than 10 tons
- Order this descending, so that it goes from smallest to largest value
- Limit this to 25 only, and see just the top countries

SELECT * FROM `project-10-sql-and-bigquery.datasets.global_gdp_co2`

WHERE CO2_emission__metric_tons_per_capita_ > 10

ORDER BY CO2_emission__metric_tons_per_capita_ DESC

LIMIT 25;

#### GDP and Mismanaged Plastic Waste of Countries with Highest CO2 Emissions 
Query Link = https://console.cloud.google.com/bigquery?sq=183187291487:bd3ad98d6f154f22949a8ae9e5d19f05

Explanation =
- Create a table where we join certain columns from gdp_co2_plastic and global_gdp_co2 
- Do the join as an inner join on the country code column
- Order descending and limit to 10 results


CREATE TABLE `project-10-sql-and-bigquery.datasets.gdp_co2_plastic` AS

SELECT 

  plastic.Entity, 

  plastic.Year, 

  gdp_co2.Total_Population,

  gdp_co2.GDP__current_US__,

  gdp_co2.GDP_per_capita__current_US__,

  gdp_co2.CO2_emission__metric_tons_per_capita_,

  plastic.Share_of_global_mismanaged_plastic_waste

FROM `project-10-sql-and-bigquery.datasets.global_gdp_co2` AS gdp_co2

INNER JOIN `project-10-sql-and-bigquery.datasets.global_plastic_management` AS plastic

ON gdp_co2.Code = plastic.Code 

ORDER BY gdp_co2.CO2_emission__metric_tons_per_capita_ DESC

LIMIT 10;

#### Now, we know how to make basic SQL queries with BigQuery!